In [17]:
import pandas as pd
import numpy as np

# 1. Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.

## Acha a quantidade de linhas de detalhes (SalesOrderDetailID) por SalesOrderID.
- Após fazer uma análise exploratória dos dados, foi verificado que **para cada SalesOrderID existe uma SalesOrderDetailID.**
 - Foi feito um agrupamento da coluna SalesOrderID com a função de agregação size() para achar o número de linhas para cada SalesOrderID, **que é o mesmo número de linhas da coluna SalesOrderDetailID.**

In [36]:
def number_of_details(file):
    
    # Importa base Sales.SalesOrderDetail.csv
    SalesOrderDetail = pd.read_csv(f"../data/{file}", sep=";")
    # Agrupa SalesOrderDetail por SalesOrderID com a função de agregação size()
    # Nomeia a coluna com o número de registros como Entries
    grouped_SalesOrderDetail = SalesOrderDetail.groupby('SalesOrderID').size().reset_index(name='Entries')
    # Seleciona as linhas com número de registros maior ou igual a 3
    answer = grouped_SalesOrderDetail.loc[grouped_SalesOrderDetail["Entries"] >= 3]
    
    # Retorna o tamanho do dataframe answer, que é o resultado esperado
    return len(answer)

In [37]:
number_of_details("Sales.SalesOrderDetail.csv")

12757